In [32]:
!pip install accelerate>=0.20.1

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
 !pip install transformers
 !pip install pytorch-lightning

In [35]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

In [36]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/data/news_summary_train_dataset.tsv', sep='\t')
df = train_df[:100]

In [37]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [38]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

model_name = 'hyunwoongko/kobart'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# import torch
# from transformers import PreTrainedTokenizerFast
# from transformers import BartForConditionalGeneration
# import torch.nn as nn

# model_name = 'hyunwoongko/kobart'
# tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [39]:
input_tokenized = []
target_tokenized = []
for i in range(len(df)):

    input_text = df.iloc[i, 0]
    target_text = df.iloc[i, 1]

    input_ids = tokenizer(input_text, padding="max_length", max_length=512, truncation=True)["input_ids"]
    target_ids = tokenizer(target_text, padding="max_length", max_length=512, truncation=True)["input_ids"]

    input_tokenized.append(input_ids)
    target_tokenized.append(target_ids)

In [40]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, input_tokenized, target_tokenized):
        self.input_tokenized = torch.LongTensor(input_tokenized)
        self.target_tokenized = torch.LongTensor(target_tokenized)

    def __len__(self):
        return len(self.input_tokenized)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_tokenized[idx],
            'labels': self.target_tokenized[idx]
        }


In [41]:
from sklearn.model_selection import train_test_split

# 학습을 위해 train, test set으로 나눈다.
input_train, input_test, target_train, target_test = train_test_split(input_tokenized, target_tokenized, test_size=0.1, random_state=42)
print(len(input_train), len(input_test), len(target_train), len(target_test))

90 10 90 10


In [42]:
train_dataset = CustomDataset(input_train, target_train)
test_dataset = CustomDataset(input_test,target_test)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model
)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [43]:
# from torch.utils.data import DataLoader

# train_dataset = CustomDataset(input_train, target_train)
# test_dataset = CustomDataset(input_test,target_test)

# dl_train = DataLoader(train_dataset, batch_size=1, shuffle=False)
# dl_test = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [44]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [45]:
# from transformers import BartConfig
# import torch.nn as nn

# class SummarizeModel(nn.Module):
#     def __init__(self, model_name = 'digit82/kobart-summarization', num_decode_layers = 1):
#         super(SummarizeModel, self).__init__()
#         self.config = BartConfig.from_pretrained(model_name)
#         self.config.decoder_layers = num_decode_layers
#         self.bart =  BartForConditionalGeneration.from_pretrained(model_name, config = self.config)
#         for param in self.bart.get_encoder().parameters():
#                 param.requires_grad = False

#     def forward(self, input_ids, dec_input):
#         result = self.bart(input_ids, dec_input)

#         return result[0]


#     def generate(self, input_ids):
#         result = self.bart(input_ids)

#         return result[0]

In [47]:
torch.cuda.empty_cache()

RuntimeError: ignored

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
from transformers import BartConfig

model_name = 'hyunwoongko/kobart'
config = BartConfig.from_pretrained(model_name)
config.decoder_layers = 3

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config = config)
for param in model.get_encoder().parameters():
    param.requires_grad = False

model

In [ ]:
model_path = "/content/drive/MyDrive/park/models/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    eval_steps=500, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
torch.save(model, "/content/drive/MyDrive/park/models/model_summary.pt")

In [ ]:
input_ids = tokenizer(df.iloc[120, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
model_path = "/content/drive/MyDrive/park/models/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=64, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps=500, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
generated_text

In [ ]:
df.iloc[120, 1]

In [ ]:
generated_text

In [ ]:
df.iloc[120, 1]

In [ ]:
from transformers import BartConfig
import torch.nn as nn

class SummarizeModel(nn.Module):
    def __init__(self, model_name = 'digit82/kobart-summarization', num_decode_layers = 1):
        super(SummarizeModel, self).__init__()
        self.config = BartConfig.from_pretrained(model_name)
        self.config.decoder_layers = num_decode_layers
        self.bart =  BartForConditionalGeneration.from_pretrained(model_name, config = self.config)
        for param in self.bart.get_encoder().parameters():
                param.requires_grad = False

    def forward(self, input_ids, dec_input):
        result = self.bart(input_ids, dec_input)

        return result[0]


    def generate(self, input_ids):
        result = self.bart(input_ids)

        return result[0]

In [ ]:
model = SummarizeModel().to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Some weights of the model checkpoint at gogamza/kobart-summarization were not used when initializing BartForConditionalGeneration: ['model.decoder.layers.5.encoder_attn.out_proj.weight', 'model.decoder.layers.4.self_attn.v_proj.weight', 'model.decoder.layers.1.self_attn_layer_norm.bias', 'model.decoder.layers.2.encoder_attn.v_proj.weight', 'model.decoder.layers.5.self_attn.v_proj.bias', 'model.decoder.layers.3.final_layer_norm.weight', 'model.decoder.layers.2.self_attn.k_proj.bias', 'model.decoder.layers.1.self_attn.q_proj.bias', 'model.decoder.layers.3.self_attn.q_proj.bias', 'model.decoder.layers.4.fc1.bias', 'model.decoder.layers.4.self_attn.out_proj.bias', 'model.decoder.layers.1.encoder_attn_layer_norm.weight', 'model.decoder.layers.4.encoder_attn.k_proj.bias', 'model.decoder.layers.4.fc1.weight', 'model.decoder.layers.4.encoder_attn.q_proj.bias', 'model.decoder.layers.3.encoder_attn.q_proj.weight', 'model.decoder.layers.1.self_attn.k_proj.weight', 'model.decoder.layers.4.encoder_

In [ ]:
model

SummarizeModel(
  (bart): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(30000, 768, padding_idx=3)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(30000, 768, padding_idx=3)
        (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
        (layers): ModuleList(
          (0-5): 6 x BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bia

In [ ]:
result = model.generate(next(iter(dl_train))['input_ids'].to(device))

In [ ]:
result.shape

torch.Size([1, 512, 30000])

In [ ]:
from tqdm import tqdm
epochs = 100
loss = nn.CrossEntropyLoss().to(device)
optim = torch.optim.Adam(params=model.parameters(), lr=0.001)

for epoch in tqdm(range(epochs)):
    epoch_loss = 0.0
    for batch in dl_train:
        input, output = batch['input_ids'].to(device), batch['labels'].to(device)
        pred = model(input, output)
        pred = pred.transpose(0, 1)
        output = output.transpose(0, 1)
        batch_loss = 0.0
        for i in range(512):  # input_size만큼 돌기
            batch_loss += loss(pred[i], output[i])
        batch_loss = batch_loss/512

        epoch_loss += batch_loss

    epoch_loss = epoch_loss/len(dl_train)
    if (epoch+1) % 10 == 0:
      print('Epoch:', '%02d' % (epoch + 1), 'cost =', '{:.6f}'.format(epoch_loss))

    optim.zero_grad()
    epoch_loss.backward()
    optim.step()

    with torch.no_grad():
        epoch_val_loss = 0.0
        for batch_val in dl_test:
            input_val, output_val = batch_val['input_ids'].to(device), batch_val['labels'].to(device)
            pred_val = model.generate(input_val)
            pred_val = pred_val.transpose(0, 1)
            output_val = output_val.transpose(0, 1)

            batch_loss_val = 0.0
            for i in range(512):
                batch_loss_val += loss(pred_val[i], output_val[i])

            batch_loss_val = batch_loss_val/512

        epoch_val_loss = epoch_val_loss/len(dl_test)
        if (epoch+1) % 10 == 0:
            print('validation loss =', '{:.6f}'.format(batch_loss_val))

  0%|          | 0/100 [00:15<?, ?it/s]


OutOfMemoryError: ignored

In [ ]:
torch.save(model, "/content/drive/MyDrive/hong/model_simple.pt")